In [1]:
import os
import csv
import gzip
import pandas as pd
import numpy as np

In [2]:
def list_files(folder_path):
    files = os.listdir(folder_path)
    return files

In [3]:
data_dir = 'C:/Users/tln229/Downloads/data process Peak/data/'

In [4]:
compress_length = 5000
compress_type = "Piecewise_Aggregation_Approximation"

for data in list_files(data_dir):
    for sample in list_files(data_dir + data + "/samples"):
        for problem in list_files(data_dir + data + "/samples/" + sample):
            for chr in list_files(data_dir + data + "/samples/" + sample + "/" + problem + "/problems"):
                sequenceID = data + "/samples/" + sample + "/" + problem + "/problems/" + chr.replace('_', ':')
                try:
                    sequence_path = data_dir + data + "/samples/" + sample + "/" + problem + "/problems/" + chr + "/coverage.bedGraph.gz"
                    extracted_file = sequence_path.replace('.gz', '')

                    # Open the compressed file and read the content
                    with gzip.open(sequence_path, 'rb') as f_in:
                        with open(extracted_file, 'wb') as f_out:
                            f_out.write(f_in.read())

                    # Load the data into a Pandas DataFrame
                    df = pd.read_csv(extracted_file, sep='\t', header=None, names=['value'])

                    # Reset the index to avoid any issues with the current index
                    df = df.reset_index(drop=True)

                    num_rows = len(df)
                    
                    # If the number of rows is greater than the target length, compress it to that length
                    if num_rows > compress_length:
                        # Determine the window size needed to reduce the data to compress_length rows
                        window_size = num_rows // compress_length
                        compact_df = df.groupby(df.index // window_size).mean().head(compress_length).reset_index(drop=True)
                    else:
                        # If the number of rows is already <= compress_length, no need to compress, just copy the data
                        compact_df = df.copy()

                    # Rename the column 'value' to 'signal'
                    compact_df.rename(columns={'value': 'signal'}, inplace=True)

                    # Add the 'sequenceID' column
                    compact_df.insert(0, 'sequenceID', sequenceID)

                    # Define the output file name
                    output_file = f'{data}/{compress_type}/{compress_length}/profiles.csv'
                    os.makedirs(os.path.dirname(output_file), exist_ok=True)

                    # Check if the file already exists to decide whether to write the header
                    file_exists = os.path.isfile(output_file)

                    # Save the compacted DataFrame to a CSV file, with header only if the file doesn't exist
                    compact_df.to_csv(output_file, mode='a', header=not file_exists, index=False)
                    
                except Exception as e:
                    print(f"An error occurred while processing {sequenceID}: {e}")

An error occurred while processing H3K36me3_TDH_immune/samples/tcell/McGill0024/problems/chr10:51448845-125869472: Error -3 while decompressing data: invalid code lengths set
